In [1]:
import time

time_start = time.time()

# loading t
train = sc.textFile("/FileStore/tables/train_ft-bfd00.txt", 2)
test = sc.textFile("/FileStore/tables/test_ft-41af5.txt", 2)
df = sc.union([train, test])

print('Time elapsed: {:.4f} seconds.'.format(time.time() - time_start))

Time elapsed: 0.0694 seconds.

In [2]:
df.take(2)

Out[ 2 ]: 
['__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^',
 "__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."]

In [3]:
# cleaning to get label numbers and reviews 
df = df.map(lambda x: x.split('__label__')[1].split(" ", 1))
print("Size of dataset: ", df.count())
df.take(2)

Size of dataset: 4000000
 Out[ 3 ]: 
[['2',
 'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'],
 ['2',
 "The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny."]]

In [4]:
# label column - distinct values
df.map(lambda x: x[0]).distinct().collect()

Out[ 4 ]: ['2', '1']

In [5]:
# convert PipelineRDD to DataFrame
df1 = df.toDF(["Label", "Review"])
df1.show(5)

+-----+--------------------+
Label| Review|
+-----+--------------------+
 2|Stuning even for ...|
 2|The best soundtra...|
 2|Amazing!: This so...|
 2|Excellent Soundtr...|
 2|Remember, Pull Yo...|
+-----+--------------------+
only showing top 5 rows

In [6]:
df1.groupby("Label").count().show()

+-----+-------+
Label| count|
+-----+-------+
 1|2000000|
 2|2000000|
+-----+-------+

In [7]:
# random split
train_set, test_set = df1.randomSplit([0.9, 0.1], seed=5)

# debug only
size = None
if size is not None:
  train_set = train_set.limit(size)
  test_set = test_set.limit(size)
  
# first two rows of train data
train_set.show(2)

# train data: label distribution
train_set.groupby("Label").count().show()

+-----+--------------------+
Label| Review|
+-----+--------------------+
 1|! Star because "0...|
 1|! WARNING! Heming...|
+-----+--------------------+
only showing top 2 rows

+-----+-------+
Label| count|
+-----+-------+
 1|1799766|
 2|1800249|
+-----+-------+

In [8]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer, StopWordsRemover, StandardScaler 
from pyspark.ml import Pipeline

# https://spark.apache.org/docs/latest/ml-features.html
time_start = time.time()

# features
tokenizer = Tokenizer(inputCol="Review", outputCol="Tokenizer")
remover = StopWordsRemover(inputCol="Tokenizer", outputCol="StopWordsRemover")
hashtf = HashingTF(numFeatures=2**16, inputCol="StopWordsRemover", outputCol='HashingTF')
idf = IDF(inputCol='HashingTF', outputCol="IDF", minDocFreq=5)

# label
label_stringIdx = StringIndexer(inputCol = "Label", outputCol = "StringIndexer")

# pipe
pipeline = Pipeline(stages=[tokenizer, remover, hashtf, idf, label_stringIdx])
pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
test_df = pipelineFit.transform(test_set)

print('Time elapsed: {:.4f} seconds.'.format(time.time() - time_start))

train_df.show(2)

Time elapsed: 113.1938 seconds.
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
Label| Review| Tokenizer| StopWordsRemover| HashingTF| IDF|StringIndexer|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
 1|! Star because "0...|[!, star, because...|[!, star, "0", un...|(65536,[1903,2803...|(65536,[1903,2803...| 1.0|
 1|! WARNING! Heming...|[!, warning!, hem...|[!, warning!, hem...|(65536,[1103,2613...|(65536,[1103,2613...| 1.0|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
only showing top 2 rows

In [9]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes

# variables
featuresCol = "IDF"
labelCol = "StringIndexer"
predictionCol = "prediction"

lr = LogisticRegression(maxIter=100, regParam=0.01, featuresCol=featuresCol, labelCol=labelCol, predictionCol=predictionCol)
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol=featuresCol, labelCol=labelCol, predictionCol=predictionCol)

models = [lr, nb]

model_results = []
for model in models:
  time_start = time.time()
  
  _fit = model.fit(train_df)
  _predictions_train = _fit.transform(train_df)
  _predictions_test = _fit.transform(test_df)
  model_results.append((_fit, _predictions_train, _predictions_test))
  
  print('Time elapsed: {:.4f} seconds.'.format(time.time() - time_start))

Time elapsed: 155.4108 seconds.
Time elapsed: 100.2385 seconds.

In [10]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

model_names = ["Logistic", "NaiveBys"]
for i, (model, predictions_train, predictions_test) in enumerate(model_results):
  time_start = time.time()
  evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol=labelCol)
  roc_auc = evaluator.evaluate(predictions_train)
  accuracy = predictions_train.filter(predictions_train.StringIndexer == predictions_train.prediction).count() / float(train_df.count())
  print("train data : model: {} roc: {:.4f} acc: {:.4f}".format(model_names[i], roc_auc, accuracy))
  print('Time elapsed: {:.4f} seconds.'.format(time.time() - time_start))
  time_start = time.time()
  roc_auc = evaluator.evaluate(predictions_test)
  accuracy = predictions_test.filter(predictions_test.StringIndexer == predictions_test.prediction).count() / float(test_df.count())
  print("test data  : model: {} roc: {:.4f} acc: {:.4f}".format(model_names[i], roc_auc, accuracy))
  print('Time elapsed: {:.4f} seconds.'.format(time.time() - time_start))

train data : model: Logistic roc: 0.9603 acc: 0.8982
Time elapsed: 252.3567 seconds.
test data : model: Logistic roc: 0.9539 acc: 0.8901
Time elapsed: 112.7529 seconds.
train data : model: NaiveBys roc: 0.4732 acc: 0.8390
Time elapsed: 256.0180 seconds.
test data : model: NaiveBys roc: 0.4713 acc: 0.8353
Time elapsed: 88.6398 seconds.

In [11]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer, StopWordsRemover, StandardScaler 
from pyspark.ml import Pipeline

# https://spark.apache.org/docs/latest/ml-features.html

time_start = time.time()

# features
tokenizer = Tokenizer(inputCol="Review", outputCol="Tokenizer")
remover = StopWordsRemover(inputCol="Tokenizer", outputCol="StopWordsRemover")
hashtf = HashingTF(numFeatures=2**16, inputCol="StopWordsRemover", outputCol='HashingTF')
idf = IDF(inputCol='HashingTF', outputCol="IDF", minDocFreq=5)
scaler = StandardScaler(inputCol="IDF", outputCol="StandardScaler", withStd=True, withMean=False)

# label
label_stringIdx = StringIndexer(inputCol = "Label", outputCol = "StringIndexer")

# pipe
pipeline = Pipeline(stages=[tokenizer, remover, hashtf, idf, scaler, label_stringIdx])
pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
test_df = pipelineFit.transform(test_set)

print('Time elapsed: {:.4f} seconds.'.format(time.time() - time_start))

train_df.show(2)

Time elapsed: 211.8254 seconds.
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
Label| Review| Tokenizer| StopWordsRemover| HashingTF| IDF| StandardScaler|StringIndexer|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
 1|! Star because "0...|[!, star, because...|[!, star, "0", un...|(65536,[1903,2803...|(65536,[1903,2803...|(65536,[1903,2803...| 1.0|
 1|! WARNING! Heming...|[!, warning!, hem...|[!, warning!, hem...|(65536,[1103,2613...|(65536,[1103,2613...|(65536,[1103,2613...| 1.0|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
only showing top 2 rows

In [12]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import LinearSVC

# variables
featuresCol = "StandardScaler"
labelCol = "StringIndexer"
predictionCol = "prediction"

lr = LogisticRegression(maxIter=100, regParam=0.01, featuresCol=featuresCol, labelCol=labelCol, predictionCol=predictionCol)
# svm = LinearSVC(maxIter=5, regParam=0.01, featuresCol=featuresCol, labelCol=labelCol)
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol=featuresCol, labelCol=labelCol, predictionCol=predictionCol)

models = [lr, nb]

model_results = []
for model in models:
  time_start = time.time()
  
  _fit = model.fit(train_df)
  _predictions_train = _fit.transform(train_df)
  _predictions_test = _fit.transform(test_df)
  model_results.append((_fit, _predictions_train, _predictions_test))
  
  print('Time elapsed: {:.4f} seconds.'.format(time.time() - time_start))

Time elapsed: 161.3334 seconds.
Time elapsed: 102.2279 seconds.

In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

model_names = ["Logistic", "NaiveBys"]
for i, (model, predictions_train, predictions_test) in enumerate(model_results):
  time_start = time.time()
  evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol=labelCol)
  roc_auc = evaluator.evaluate(predictions_train)
  accuracy = predictions_train.filter(predictions_train.StringIndexer == predictions_train.prediction).count() / float(train_df.count())
  print("train data : model: {} roc: {:.4f} acc: {:.4f}".format(model_names[i], roc_auc, accuracy))
  print('Time elapsed: {:.4f} seconds.'.format(time.time() - time_start))
  time_start = time.time()
  roc_auc = evaluator.evaluate(predictions_test)
  accuracy = predictions_test.filter(predictions_test.StringIndexer == predictions_test.prediction).count() / float(test_df.count())
  print("test data  : model: {} roc: {:.4f} acc: {:.4f}".format(model_names[i], roc_auc, accuracy))
  print('Time elapsed: {:.4f} seconds.'.format(time.time() - time_start))

train data : model: Logistic roc: 0.9603 acc: 0.8982
Time elapsed: 269.1483 seconds.
test data : model: Logistic roc: 0.9539 acc: 0.8901
Time elapsed: 103.4503 seconds.
train data : model: NaiveBys roc: 0.4794 acc: 0.8070
Time elapsed: 252.2488 seconds.
test data : model: NaiveBys roc: 0.4774 acc: 0.7992
Time elapsed: 85.8507 seconds.

In [14]:
from pyspark.sql.functions import *

# logistic regression as best model
best_model = model_results[0][0]

# roc curve
display(best_model, test_df.select(col("StandardScaler"), col("StandardScaler").alias("features"), col("StringIndexer")), 'ROC')

False Positive Rate,True Positive Rate,Threshold
0.0,0.0,0.9999352600302102
0.0,0.016129032258064516,0.9999352600302102
0.0,0.03225806451612903,0.9998652601457332
0.0,0.04838709677419355,0.9998206422532102
0.0,0.06451612903225806,0.9994485293892199
0.0,0.08064516129032258,0.9994364689089831
0.0,0.0967741935483871,0.9993897791570268
0.0,0.11290322580645161,0.9992944860033893
0.0,0.12903225806451613,0.9992822631375041
0.0,0.14516129032258066,0.9985250605257556


In [15]:
display(best_model, test_df.select(col("StandardScaler"), col("StandardScaler").alias("features"), col("StringIndexer")))

fitted values,residuals
-0.5903940936076832,0.6434555639580033
1.8366971962379186,0.13744237780678992
-8.620425483704837,-1.8035095580142552E-4
-2.54230816716807,-0.07294493205747529
-4.313898428881058,-0.013204587497452677
1.8388357815712222,-0.8628109591747265
-2.2002484940291334,-0.09972817649342447
7.255878184601409,7.055139966106738E-4
-0.8480607975268599,-0.29983980765178087
-5.166274566158245,-0.005673414512995215
